# ASSIGNMENT_2_1_RT - INTERACTIVE CLIENT NODE

### Import and ROS initialization

This section sets up the core components required for integrating ROS with the Jupyter environment. It imports necessary libraries, initializes global variables, and prepares the system for robot control, real-time data handling, and interaction through widgets and plots. This foundational setup enables communication with ROS topics, services, and actions, and is essential for the rest of the notebook to function correctly.

In [ ]:
import rospy
import time
import actionlib
import sys
import math
from sensor_msgs.msg import LaserScan
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Pose, Twist, PoseStamped
from actionlib_msgs.msg import GoalStatus

from assignment2_1_rt.msg import PosVel, PlanningAction, PlanningGoal

import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import HBox, VBox

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation as FAnim

#______________________REQUIRED GLOBAL VARIABLES______________________
#Standard required global variables
OdPose = Pose()
OdTwist = Twist()
client = None
pub_PsVl = None

min_obstacle_distance = widgets.FloatText(value=0.0, description='closest obstacle [m]:', disabled=True)

#Output plot global variables
goal_output = widgets.Output()
output_ClientServer = widgets.Output()

### Odom callback and publisher

This section defines the callback functions for handling odometry data and periodically publishing the robot's position and velocity. It ensures the system stays updated with the robot's current state and makes this information available to other ROS components through a custom message. Scan callback is defined in order to provide the distance from the closest obstacle, based on 

In [ ]:
def odom_callback(msg):
    global OdPose, OdTwist
    OdPose = msg.pose.pose
    OdTwist = msg.twist.twist

def pos_callback(event=None):
    global OdPose, OdTwist, pub_PsVl
    if pub_PsVl:
        msg = PosVel()
        msg.pos_x = OdPose.position.x
        msg.pos_y = OdPose.position.y
        msg.vel_x = OdTwist.linear.x
        msg.vel_z = OdTwist.angular.z
        pub_PsVl.publish(msg)
        
def scan_callback(msg):
    global min_obstacle_distance
    
    valid_ranges = [r for r in msg.ranges if r > 0.01 and not rospy.is_shutdown() and not (math.isinf(r))]
    if valid_ranges:
        min_obstacle_distance.value = min(valid_ranges)
    else:
        min_obstacle_distance.value = None

### Send and cancel goal functions for goal management

This section defines the functions to send and cancel navigation goals via the action client. When a new goal is sent, the target position is taken from the input widgets and transmitted to the planner. If a goal is active, it can also be canceled, providing interactive control over the robot’s behavior directly from the notebook interface.

In [ ]:
def feedback_clbk(feedback):
    global reached_count
    with goal_output:
        if (feedback.stat == "Target reached!"):
            reached_count += 1
            clear_output(wait=True)
            print("Target reached\n{}\nStatus: {}\n".format(feedback.actual_pose, feedback.stat))

def send_goal(_):
    global client
    with goal_output:
        clear_output(wait=True)
        if not client.get_state() in [GoalStatus.ACTIVE, GoalStatus.PENDING]:

            target = PoseStamped()
            target.pose.position.x = x_input.value
            target.pose.position.y = y_input.value

            goal = PlanningGoal()
            goal.target_pose = target

            client.send_goal(goal, feedback_cb = feedback_clbk)
            print(f"Sent goal: ({target.pose.position.x}, {target.pose.position.y})")
        else:
            print("Goal already sent")
            
def cancel_goal(_):
    global client, failed_count
    with goal_output:
        clear_output(wait=True)
        if client.get_state() in [GoalStatus.ACTIVE, GoalStatus.PENDING]:
            client.cancel_goal()
            rospy.sleep(1)
            state = client.get_state()
            if state in [GoalStatus.PREEMPTED, GoalStatus.RECALLED]:
                print('Goal canceled')
                failed_count += 1
            else:
                print("Can't clear current goal")
        else:
            print("No active goal")

### ROS node initialization function

This section initializes the ROS node and sets up communication with the rest of the ROS ecosystem. It subscribes to odometry data, advertises a publisher for custom position and velocity messages, and connects to the action server used for goal planning. A timer is also started to periodically publish the robot’s state.

In [ ]:
def init_ros_node():
    global client, pub_PsVl
    rospy.init_node('interactive_action_client')
    
    pub_PsVl = rospy.Publisher("/posVel", PosVel, queue_size=10)
    
    with output_ClientServer:
        rospy.loginfo("---Checking '\odom' topic---")
    rospy.wait_for_message('/odom', Odometry)
    rospy.Subscriber("/odom", Odometry, odom_callback)
    
    with output_ClientServer:
        rospy.loginfo("---Checking '\scan' topic---")
        rospy.wait_for_message("/scan", LaserScan)
    rospy.Subscriber("/scan", LaserScan, scan_callback)
    
    client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
    with output_ClientServer:
        rospy.loginfo("---Waiting for action server---")
        
    client.wait_for_server()
    with output_ClientServer:
        rospy.loginfo("---Action server connected---")
        rospy.loginfo("---SYSTEM READY---")
    rospy.Timer(rospy.Duration(0.5), pos_callback)

        
try:
    init_ros_node()
except rospy.ROSInterruptException:
    print("ROS Interrupt Exception detected")

In [ ]:
display(output_ClientServer)

### Interactive widgets for control purposes

This section creates interactive input fields and buttons using `ipywidgets` to allow the user to send or cancel navigation goals. The X and Y fields define the target position, while the "Send Goal" and "Cancel Goal" buttons trigger the corresponding ROS actions. Output feedback is displayed below the controls.

In [ ]:
x_input = widgets.FloatText(value=0.0, description='X:', step=0.1)
y_input = widgets.FloatText(value=0.0, description='Y:', step=0.1)

send_button = widgets.Button(description='Send Goal', button_style='success')
cancel_button = widgets.Button(description='Canceal Goal', button_style='danger')

display(widgets.HBox([x_input, y_input]))
display(widgets.HBox([send_button, cancel_button]))

send_button.on_click(send_goal)
cancel_button.on_click(cancel_goal)

In [ ]:
display(goal_output)

### Position live check

This section creates an interactive plot to visualize the robot's position and its movement trajectory over time. The robot's position is updated in real-time, while the trajectory can be optionally enabled or disabled through a dropdown menu. Additionally, the "Reset" button allows for clearing the trace. The plot dynamically updates based on the robot's odometry data, providing an interactive visualization of the robot's movement. Key features:
- **Trace Control**: The dropdown menu allows switching between keeping the trace of the robot's movement or disabling it.
- **Reset Trace**: The "Reset" button clears the current trajectory trace on the plot.
- **Real-time Updates**: The robot's current position is updated every 0.5 seconds and plotted on the graph.

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots()
robot_pos, = ax.plot([], [], 'ro', label='Robot')
trace_line, = ax.plot([], [], 'b-', alpha=0.5, label='Trace')

ax.set_xlim(-12, 12)
ax.set_ylim(-12, 12)
ax.grid(True)
ax.legend()
ax.set_xlabel("X")
ax.set_ylabel("Y")

# Global variables used for the trace
trace_enabled = True
trace_visible = True
trace_x = []
trace_y = []

def on_trace_mode_change(change): # Dropdown menu definition
    global trace_enabled, trace_visible
    mode = change['new']
    if mode == 'Keep trace':
        trace_enabled = True
        trace_visible = True
    elif mode == 'Disable trace':
        trace_enabled = False
        trace_visible = False
        trace_line.set_data([], [])
        
def reset_trace(_): # Reset button function to clean the trace
    global trace_x, trace_y
    trace_x.clear()
    trace_y.clear()
    trace_line.set_data([], [])

def init_function():
    trace_line.set_data([],[])
    robot_pos.set_data([],[])
    return (trace_line, robot_pos)

def update_trace(frame):
    global trace_enabled, trace_visible, trace_x, trace_y, ax, robot_pos
    
    x = OdPose.position.x
    y = OdPose.position.y
    robot_pos.set_data(x, y) #to plot position
    
    ax.set_title(f"Robot Position: x={x:.2f}, y={y:.2f}") #to display current position
    
    if trace_enabled:
            trace_x.append(x)
            trace_y.append(y)
    if trace_visible:
        trace_line.set_data(trace_x, trace_y) #to plot trace
    
    ax.relim()
    ax.autoscale_view()
    return (trace_line, robot_pos)

trace_mode_dropdown = widgets.Dropdown( # Dropdown menu init
    options=['Keep trace','Disable trace'],
    value='Keep trace',
    description='Trace:'
)

reset_button = widgets.Button(description='Reset', button_style='info') #trace-reset button init

trace_mode_dropdown.observe(on_trace_mode_change, names='value') # Connect callbacks
reset_button.on_click(reset_trace)

display(widgets.HBox([trace_mode_dropdown, reset_button])) # Display widgets in a clear HBox

RobPosAnim = FAnim(fig, update_trace, init_func=init_function, interval=500, blit=True)

### Reached/not-reached targets count

In [ ]:
#Variables for reached/non-reached targets
reached_count, failed_count = 0, 0

fig_bars, ax_bars = plt.subplots()
bars = ax_bars.bar(['Reached', 'Not reached'], [0, 0], color=['blue', 'red'])
ax_bars.set_title("Goal outcomes")
ax_bars.set_ylim(0, max(5, reached_count + reached_count + 1))

def update_bars(frame):
    bars[0].set_height(reached_count)
    bars[1].set_height(failed_count)
    tot = reached_count + failed_count
    ax_bars.set_ylim(0, max(5, tot + 1))
    return bars

count_bars_plot = FAnim(fig_bars, update_bars, interval=500, blit=True)

### Laser scan - check distance

This cell sets up a live display showing the distance from the robot to the closest obstacle detected by its laser scanner. A label widget is used to present this distance in real time. The program subscribes to the */scan* topic, which provides LaserScan messages containing an array of range values. Each time a message is received, the callback function filters out invalid readings (such as zero, infinity, or NaN) and determines the closest valid distance by taking the minimum. A separate thread continuously updates the label on the notebook interface every half second, displaying the most recent distance to the nearest obstacle. If no valid distance is available, the label shows a placeholder value. This allows users to visually monitor how close the robot is to surrounding objects during operation.

In [ ]:
display(min_obstacle_distance)